## Libraries

In [ ]:
!pip install langchain langchain_core langchain_community langchain-huggingface torch accelerate bitsandbytes docarray unstructured

## Requirements

Install the requirements and import relevant modules

In [5]:
import os
import pandas as pd
from pandas import DataFrame

## API Keys and Environment Variables

Set the API keys and environment variables required for running the app

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
os.environ['LANGCHAIN_API_KEY'] = ""
os.environ["OPENAI_API_KEY"] = ""

# The hugging face cahces is the location
# where the models will be downloaded. It is 
# recommended to set it in a location which has
# sufficient storage space

os.environ["HF_HUB_CACHE"] = ""
os.environ["HF_HOME"] = ""

## Data Loaders

Functions to load data from excel files

In [26]:
def load_remediations_to_dataframe(file_path: str, header, index_col, reset_index=False) -> DataFrame:
    """
    Loads the remediation table into a pandas dataframe.

    Args:
        * file_path (str): The path to the remidations excel file

    Returns:
        A pandas `DataFrame` object with the remediation data
    """

    df = pd.read_excel(file_path, header=header, index_col=index_col)

    if reset_index:
        df = df.reset_index()
    return df

In [ ]:
df = load_remediations_to_dataframe("./data/Remediations.xlsx", header=2, index_col=[0,1,2,3,4], reset_index=True)